<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [3]:
import pandas as pd

yelp = pd.read_json('./sprint_data/review_sample.json', lines=True)

In [4]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


(10000, 9)

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [45]:
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_sm")

def get_lemmas(text):
    lemmas = []
    
    doc = nlp(text)
    
    for token in doc:
        if ((token.is_stop == False) and (token.is_punct == False) and token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
            
    
    return lemmas

In [46]:
yelp['tokens'] = yelp['text'].apply(get_lemmas)

In [87]:
yelp['tokens'][:5]

0    absolute bad place stay 43 year life   time ti...
1    absolute bad place stay 43 year life   time ti...
2    absolute bad place stay 43 year life   time ti...
3    absolute bad place stay 43 year life   time ti...
4    absolute bad place stay 43 year life   time ti...
Name: tokens, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [57]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [78]:
strings = []
for token in yelp['tokens']:
    strings.append([' '.join(token)])

In [88]:
for token in range(yelp['tokens'].shape[0]):
    yelp['tokens'][token:token+1] = strings[token][0]

/Users/lambda_school_loaner_95/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [90]:
yelp['tokens'][-5:]

9995    family hungry Subway open 24 hour guy work say...
9996    wife come couple friend sever excited point ob...
9997    food ok brag food hot item tasty horrible look...
9998    today visit great love enjoy Town Square Las V...
9999    absolute bad place stay 43 year life   time ti...
Name: tokens, dtype: object

In [91]:
vectorizer = CountVectorizer(stop_words='english')
# can ad custom tokenizer as long as it returns a list

# tokenize and build vocab
vectorizer.fit(yelp['text'])

# Create a Vocabulary
# The vocabulary establishes all of the possible words that we might use.
vectorizer.vocabulary_

# The vocabulary dictionary does not represent the counts of words!!
dtm = vectorizer.transform(yelp['text'])

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features = 5000, min_df=.05)

# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(yelp['tokens'])

In [101]:
nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')

# Fit on TF-IDF Vectors
nn.fit(dtm.todense())

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [102]:
fake_review = ["""
I love Crystal Creek. Their drinks are great, their vibe is dope, and it's such a fun environment. 
The bartenders are super fun and the bouncer, Rocket, makes sure everyone's being respectful 
(see picture of Rocket  attached). This is a cool place out in Spicewood to end the evening on.
"""]

In [103]:
random = tfidf.transform(fake_review)

nn.kneighbors(random.todense())

(array([[0.83899653, 0.88507658, 0.94390376, 0.95260172, 0.95844194,
         0.96408191, 0.96408191, 0.96408191, 0.9956159 , 0.99663324]]),
 array([[5184, 7196, 8197, 2684, 2295, 7955, 4600, 6278, 7471, 2286]]))

In [107]:
## print text of reviews
print(yelp['tokens'][5184], '\n\n', yelp['tokens'][7196], '\n\n', yelp['tokens'][8197], '\n\n',
      yelp['tokens'][2684], '\n\n', yelp['tokens'][2295], '\n\n', yelp['tokens'][7955], '\n\n',
      yelp['tokens'][4600], '\n\n', yelp['tokens'][6278], '\n\n', yelp['tokens'][7471], '\n\n',
      yelp['tokens'][2286])

Love Kabobs way lamb super tasty great middle eastern flavor beef stew soup good end spoon rice 

 hungry forgot picture 

 Super yummy sandwich person great judge sandwich city love family own sandwich place great quality 

 main reason drop place absolutely obsess penguin enter initial thought wow expect waiter waitress super earth tattoo ton piercing earth attitude confuse give bad review opinion absolutely love atmosphere place great sit hang friend family stranger people super friendly helpful food great boyfriend express love cheeseburger great place stop need STAT 

 bar cram clean waitress super friendly accommodate   sure year bloody mary super great 

 great tan Z Bods weekend Abby tan perfectly personal will tan sure soon 

 calm beautiful place god   great message magnificent music add joy beautiful surrounding   love bookstore 

 great place huge portion love bear stuff brunch mom daughter 

 great hidden gem Hudson   love place   southern cooking dome flair   favs Hudson 

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [110]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(yelp['tokens'], yelp['stars'], test_size=.33, random_state=42)

vect = TfidfVectorizer(stop_words='english')
sgdc = SGDClassifier()
pipe = Pipeline([('vect', vect), ('classifier', sgdc)])

#Fit pipeline:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [113]:
accuracy_score(y_test, y_pred)

0.5951515151515151

### Tuning with Grid Search

In [144]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'classifier__max_iter': (20, 10, 100)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=10)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done  43 out of  45 | elapsed:   28.7s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   29.2s finished
/Users/lambda_school_loaner_95/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'classifier__max_iter': (20, 10, 100)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [145]:
best = grid_search.best_estimator_
grid_y_pred = best.predict(X_test)

In [146]:
accuracy_score(y_test, grid_y_pred)

0.5990909090909091

In [115]:
yelp['stars'].value_counts(normalize=True)

5    0.4462
4    0.2185
1    0.1496
3    0.1098
2    0.0759
Name: stars, dtype: float64

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [116]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import matplotlib.pyplot as plt

In [117]:
def gather_data(reviews):
    
    data = []
    
    for r in reviews: 
        
        tokens = tokenize(str(reviews))
        data.append(tokens)
    
    return data

In [182]:
data = gather_data(yelp['tokens'])

In [183]:
id2word = corpora.Dictionary(data)

In [180]:
id2word.filter_extremes(no_below=5, no_above=0.90)

In [184]:
corpus = [id2word.doc2bow(review) for review in data]

In [186]:
len(corpus[0][0])

2

In [179]:
lda = LdaMulticore(corpus=corpus,
                  id2word=id2word,
                  random_state=42,
                  num_topics=10,
                  workers=4
                  )

In [134]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [135]:
topics = [' '.join(t[:3]) for t in words]

In [136]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [137]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/Users/lambda_school_loaner_95/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.003244  0.002892       1        1  11.031054
3      0.002017  0.000264       2        1  10.775642
0      0.000098  0.000201       3        1  10.405326
1      0.000511 -0.000680       4        1  10.310887
7      0.001759 -0.002470       5        1  10.291472
6     -0.001727 -0.000803       6        1   9.695089
9     -0.000376 -0.001116       7        1   9.685155
2     -0.002817 -0.001483       8        1   9.500224
8      0.000197 -0.000627       9        1   9.315922
4     -0.002905  0.003822      10        1   8.989231, topic_info=    Category          Freq        Term         Total  loglift  logprob
143  Default  69587.000000       great  69587.000000  30.0000  30.0000
32   Default  51283.000000        \n\n  51283.000000  29.0000  29.0000
281  Default  68687.000000     service  68687.000000  28.0000  28.0000
141  Default  98061.000000        good  98061.000000  27.0000  27.0000
205  Default  30973.000000        menu  30973.000000  26.0000  26.0000
319  Default  60028.000000        time  60028.000000  25.0000  25.0000
130  Default  30303.000000       fresh  30303.000000  24.0000  24.0000
37   Default  50393.000000     amazing  50393.000000  23.0000  23.0000
126  Default  69089.000000        food  69089.000000  22.0000  22.0000
352  Default  29565.000000        year  29565.000000  21.0000  21.0000
1    Default  20046.000000          10  20046.000000  20.0000  20.0000
194  Default  40178.000000        love  40178.000000  19.0000  19.0000
218  Default  30592.000000       night  30592.000000  18.0000  18.0000
51   Default  20497.000000        beef  20497.000000  17.0000  17.0000
209  Default  10493.000000    montreal  10493.000000  16.0000  16.0000
140  Default  50120.000000          go  50120.000000  15.0000  15.0000
115  Default  30455.000000        fake  30455.000000  14.0000  14.0000
341  Default  19641.000000       watch  19641.000000  13.0000  13.0000
273  Default  20520.000000         say  20520.000000  12.0000  12.0000
196  Default  20112.000000       lunch  20112.000000  11.0000  11.0000
263  Default  20345.000000  restaurant  20345.000000  10.0000  10.0000
220  Default  20027.000000      noodle  20027.000000   9.0000   9.0000
101  Default  20299.000000       drink  20299.000000   8.0000   8.0000
340  Default  20310.000000        walk  20310.000000   7.0000   7.0000
279  Default  10359.000000       serve  10359.000000   6.0000   6.0000
77   Default  20102.000000      circus  20102.000000   5.0000   5.0000
129  Default   9829.000000       forum   9829.000000   4.0000   4.0000
65   Default  19602.000000        busy  19602.000000   3.0000   3.0000
136  Default  49947.000000         get  49947.000000   2.0000   2.0000
107  Default  19873.000000         end  19873.000000   1.0000   1.0000
..       ...           ...         ...           ...      ...      ...
176  Topic10   1113.678101       kraft  10012.062500   0.2130  -5.9699
53   Topic10   1116.870850      beware  10081.930664   0.2089  -5.9671
327  Topic10   1116.001099        tour  10081.036133   0.2082  -5.9678
101  Topic10   2203.790039       drink  20299.726562   0.1887  -5.2874
205  Topic10   3266.273682        menu  30973.248047   0.1597  -4.8939
61   Topic10   2154.761719      buffet  20147.669922   0.1737  -5.3099
136  Topic10   4973.789551         get  49947.074219   0.1024  -4.4734
126  Topic10   6577.005859        food  69089.265625   0.0573  -4.1940
194  Topic10   4037.362305        love  40178.671875   0.1114  -4.6820
319  Topic10   5502.833496        time  60028.609375   0.0196  -4.3723
108  Topic10   2091.529541       enjoy  19851.662109   0.1588  -5.3397
44   Topic10   2080.649658     awesome  19794.197266   0.1564  -5.3449
141  Topic10   7955.876465        good  98061.937500  -0.1025  -4.0037
143  Topic10   5667.477539       great  69587.070312  -0.0987  -4.3428
140  Topic10   4226.563965          go  501

In [138]:
distro = [lda[d] for d in corpus]

In [139]:
def update(doc):
    d_dist = {k:0 for k in range(0,3)}
    for t in doc:
        d_dist[t[0]] = t[1]
    return d_dist

new_distro = [update(d) for d in distro]

In [141]:
df_distro = pd.DataFrame.from_records(new_distro)
df_distro.columns = topics
df_distro['stars'] = yelp['stars']

In [142]:
df_distro.head()

,good food time,good great food,service great food,good service great,good \n\n food,good food great,good great time,good service place,good great service,good food service,stars
0,0.095442,0.134489,0.099430,0.081039,0.109884,0.114156,0.103827,0.099038,0.088693,0.074003,1
1,0.102073,0.134879,0.096491,0.115645,0.039926,0.101781,0.084390,0.096708,0.105131,0.122977,4
2,0.106589,0.084181,0.090468,0.108731,0.111972,0.122389,0.079082,0.098128,0.146918,0.051542,3
3,0.054478,0.138235,0.116778,0.114537,0.067033,0.121999,0.070710,0.091003,0.105376,0.119850,1
4,0.125981,0.044595,0.103188,0.099606,0.079684,0.105309,0.091549,0.148051,0.112323,0.089714,4


In [143]:
df_distro.groupby('stars').mean().head()

,good food time,good great food,service great food,good service great,good \n\n food,good food great,good great time,good service place,good great service,good food service
stars,,,,,,,,,,
1,0.103883,0.104629,0.095179,0.108542,0.090202,0.109530,0.095507,0.102837,0.092568,0.097123
2,0.105031,0.103065,0.094925,0.107806,0.089944,0.109861,0.097623,0.100653,0.093897,0.097196
3,0.103895,0.102713,0.094016,0.109264,0.090945,0.110146,0.096430,0.103863,0.091964,0.096765
4,0.105201,0.102841,0.095054,0.107802,0.090335,0.109521,0.097325,0.103037,0.093033,0.095852
5,0.103516,0.103353,0.095876,0.108538,0.089653,0.110219,0.096728,0.103188,0.092463,0.096465


### Analysis

I can tell from my topic model that I need to add some specific stop words to make the results more meaningful.  "Good", "Service", "Food", "Great" come to mind.  For some reason the filtering of high frequency words did not work on the id2word function.. which is another reason.  I ran out of time before I could debug it. 

Despite all that, I have a frame work for analyzing the text pretty well.  I was also able to group the tf-idf dataframe by rating.  By doing this, and better filtering of words, I feel like it would be possible to get a sentiment on what time of things makes up a five star review.  By doing so, a business can make use of feedback more efficiently.  The same can be said for low star reviews.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)